In [63]:
# ## Import packages
import datetime
import io
import math
import os
import shutil
import re
import warnings
from datetime import datetime, timedelta, timezone

import numpy as np
import pandas as pd
import requests
from pytz import timezone
from tqdm.autonotebook import tqdm

In [64]:
# ## Setup from Info.xlsx
info = pd.read_excel("Info/Info.xlsx", sheet_name=None, index_col=0)

In [65]:
# ## Functions
# Date Functions
def set_date(date_str, old_tz, dt_format="%d/%m/%Y %H:%M:%S"):
    if date_str == 'NaT':
        return pd.NaT
    else:
        datetime_set_naive = datetime.strptime(date_str, dt_format)
        datetime_set_old = timezone(old_tz).localize(datetime_set_naive)
        datetime_set_utc = datetime_set_old.astimezone(timezone('UTC'))
        return datetime_set_utc

def date_parser(date_, time_, dt_format="%d/%m/%Y %H:%M:%S"):
    return set_date(date_ + " " + time_, 'UTC', dt_format)


def date_parserYMD(date_, time_, dt_format="%Y-%m-%d %H:%M:%S"):
    return set_date(date_ + " " + time_, 'UTC', dt_format)


# Experiment start/end date
date_start = set_date(str(info['setup'].loc['date_start_utc', 'value']), "UTC", "%Y-%m-%d %H:%M:%S")
if pd.isna(info['setup'].loc['date_end_utc', 'value']):
    date_now = datetime.now(timezone('UTC'))
    info['setup'].loc['date_end_utc', 'value'] = date_now
else:
    date_now = set_date(str(info['setup'].loc['date_end_utc', 'value']), "UTC", "%Y-%m-%d %H:%M:%S")


# Date Functions continued
def date_range(window=-1, start=date_start, end=date_now):
    if pd.isna(start):
        start = date_start
    if pd.isna(end):
        end = date_now
    if window != -1:
        if not pd.isna(window):
            start = end - timedelta(days=window)
    return start, end


# ## Process Information sheets from Info.xlsx
# ### Chartxs:
charts = []
for chart, row in info['charts'].iterrows():
    chart_range = date_range(window=row['chart_range_window'],
                             start=set_date(str(row['chart_range_start']), "UTC", "%Y-%m-%d %H:%M:%S"),
                             end=set_date(str(row['chart_range_end']), "UTC", "%Y-%m-%d %H:%M:%S"))
    info['charts'].loc[chart, 'chart_range_start'] = chart_range[0]
    info['charts'].loc[chart, 'chart_range_end'] = chart_range[1]
    info['charts'].loc[chart, 'chart_range_window'] = chart_range[1] - chart_range[0]
    charts.append(chart)

del chart, chart_range, row

# ### Colours:

info['colours']['rgb'] = list(
    zip((info['colours']['r'] / 255), (info['colours']['g'] / 255), (info['colours']['b'] / 255)))
info['colours']['rgba_str'] = "rgba(" + info['colours']['r'].astype(int).astype(str) + "," + info['colours']['g'].astype(
    int).astype(str) + "," + info['colours']['b'].astype(int).astype(str) + ",1)"


# ## Import Data
# ### Custom Support Data Functions

def get_UO_support_data(dataset_f_info, dataset="UO"):
    def import_UO_log_dates():
        UO_log_dates = pd.read_csv(dataset_f_info['supp_data_filepath'][dataset], sep="\t")
        UO_log_dates['start'] = pd.to_datetime(UO_log_dates['start'], format="%Y-%m-%d")
        UO_log_dates['end'] = pd.to_datetime(UO_log_dates['end'], format="%Y-%m-%d")
        return UO_log_dates

    UO_log_dates = import_UO_log_dates()

    # Check LogDates file
    if max(UO_log_dates['end']) < date_now.date() - timedelta(days=1):
        # print("Retrieving latest Urban Observatory data...")
        # api-endpoint
        URL = "http://uoweb3.ncl.ac.uk/api/v1.1/sensors/data/csv/?"
        # location given here
        poly = '0103000000010000000600000001000020a2baf9bf58094ee3a2764b4001000020f268fbbf90c059b5b5714b4000000020ba37fbbfc4e78445c6704b4001000020b203fbbf9ce8d60e68714b40010000209a59f9bf346a0efd06714b4001000020a2baf9bf58094ee3a2764b40'
        # defining a params dict for the parameters to be sent to the API
        PARAMS = {'polygon_wkb': poly,
                  'starttime': max(UO_log_dates['end']).strftime("%Y%m%d%H%M%S"),  # "%Y%m%d%H%M%S"
                  'endtime': date_now.strftime("%Y%m%d%H%M%S"),  # "%Y%m%d%H%M%S"
                  'data_variable': 'River Level,Temperature,Rainfall'}
        # sending get request and saving the response as response object
        data_content = requests.get(url=URL, params=PARAMS).content
        UO_data = pd.read_csv(io.StringIO(data_content.decode("utf-8")))
        min_date = pd.to_datetime(min(UO_data['Timestamp'])).strftime("%Y-%m-%d")
        max_date = pd.to_datetime(max(UO_data['Timestamp'])).strftime("%Y-%m-%d")

        if pd.to_datetime(max_date) > max(UO_log_dates['end']):
            UO_data.to_csv(dataset_f_info['data_folder_path'][dataset] + "UO_data_" + str(min_date) + "_to_" + str(
                max_date) + ".csv",
                           index=False)
            UO_Files = []
            for filename in tqdm(os.listdir(str(dataset_f_info['data_folder_path'][dataset])),
                                 desc="Open existing UO files"):
                if re.search(dataset_f_info['file_pat'][dataset], filename) and not filename.startswith('.'):
                    UO_Files.append(filename)
            UO_starts = []
            for string in UO_Files: UO_starts.append(string[8:18])
            UO_ends = []
            for string in UO_Files: UO_ends.append(string[22:32])

            new_UO_file_data = {'file': UO_Files,
                                'start': UO_starts,
                                'end': UO_ends}
            UO_log_dates_new = pd.DataFrame(new_UO_file_data, columns=['file', 'start', 'end'])
            UO_log_dates_new.to_csv(dataset_f_info['supp_data_filepath'][dataset], index=False, sep="\t")
            print("Urban Observatory data updated!")
        else:
            print("Urban Observatory data up-to-date!")

    UO_log_dates = import_UO_log_dates

    return UO_log_dates


def get_Events_support_data(dataset_f_info, dataset="Events"):
    Events_info = pd.read_csv(dataset_f_info['supp_data_filepath'][dataset], sep="\t", index_col="Code")
    return Events_info


custom_preimport_functions = {'UO': get_UO_support_data,
                              'Events': get_Events_support_data}


# ### Custom Imported Data Functions
def mod_imported_Sensor_py_data(day_df, filename, dataset_supp_data, dataset_f_info, dataset="Sensor"):
    day_df['DateTime'] = day_df['Date'] + " " + day_df['Time']
    day_df['DateTime'] = pd.to_datetime(day_df['DateTime'], format='%d/%m/%y %H:%M:%S')
    day_df['DateTime'] = day_df['DateTime'].dt.tz_localize('UTC')

    # Current density
    day_df.iloc[:, day_df.columns.str.contains("__C")] = day_df.iloc[:, day_df.columns.str.contains("__C")] / (
            math.pi * 0.6 ** 2 * 10)
    # Sum Current density
    day_df.iloc[:, day_df.columns.str.contains("SUM__C")] = day_df.iloc[:, day_df.columns.str.contains("SUM__C")] / 4

    return day_df


def mod_imported_Skid_data(day_df, filename, dataset_supp_data, dataset_f_info, dataset="Skid"):
    day_df = day_df.rename(columns={"TIME": "DateTime"})
    day_df['DateTime'] = pd.to_datetime(day_df['DateTime'], format='%Y/%m/%d %H:%M:%S.%f')
    day_df['DateTime'] = day_df['DateTime'].dt.tz_localize('UTC')
    selected_pars = list(info['parameters'][selected].query('dataset == "' + dataset + '"')['parameter'].values)
    day_df = day_df[day_df[selected_pars].sum(axis=1, skipna=True) != 0]
    return day_df


def mod_imported_SampLog_data(day_df, filename, dataset_supp_data, dataset_f_info, dataset="SampLog"):
    day_df = day_df.rename(columns={"Date": "DateTime"})
    day_df['DateTime'] = pd.to_datetime(day_df['DateTime'], format='%d/%m/%Y %H:%M')
    day_df['DateTime'] = day_df['DateTime'].dt.tz_localize('Europe/London')
    day_df['DateTime'] = day_df['DateTime'].dt.tz_convert('UTC')

    selected_read_cols = ['R1', 'R2', 'R3']
    day_df[selected_read_cols] = day_df[selected_read_cols].apply(pd.to_numeric, errors='coerce')

    # Average readings
    day_df['Read_ave'] = day_df[selected_read_cols].mean(axis=1, skipna=True)

    # Widen DF
    day_df_wide = pd.pivot_table(day_df, values='Read_ave', index=['DateTime', 'Location'], columns=['Type', 'Vial'])
    # Fix col header and names
    day_df_wide.columns = map(''.join, (str(v) for v in day_df_wide.columns))
    day_df_wide.columns = [re.sub(r'\W', '', i) for i in day_df_wide.columns]
    day_df_wide.columns = [s[:len(s) - 1] + "_" + s[len(s) - 1:] for s in day_df_wide.columns]
    day_df_wide = day_df_wide.reset_index()

    return day_df_wide


def mod_imported_NWL_data(day_df, filename, dataset_supp_data, dataset_f_info, dataset="NWL"):
    day_df['DateTime'] = day_df['DATE'].astype(str) + " " + day_df['TIME']
    day_df['DateTime'] = pd.to_datetime(day_df['DateTime'], format='%Y-%m-%d %H:%M')
    day_df['DateTime'] = day_df['DateTime'].dt.tz_localize('Europe/London')
    day_df['DateTime'] = day_df['DateTime'].dt.tz_convert('UTC')

    day_df = day_df[day_df['SITE'].str.contains('SETTLED')]

    return day_df


def mod_imported_UO_data(day_df, filename, dataset_supp_data, dataset_f_info, dataset="UO"):
    day_df['DateTime'] = pd.to_datetime(day_df['Timestamp'], format='%Y-%m-%d %H:%M:%S')
    day_df['DateTime'] = day_df['DateTime'].dt.tz_localize('UTC')

    # Create Type column
    day_df['Type'] = day_df['Sensor Name'].astype(str) + "_" + day_df['Variable']

    # Widen DF
    day_df_wide = pd.pivot_table(day_df, values='Value', index=['DateTime'], columns=['Type'])
    # Fix col header and names
    day_df_wide = day_df_wide.reset_index()
    day_df_wide = day_df_wide.rename(columns=dict(zip(info['parameters'].query('dataset == "' + dataset + '"')['code'],
                                                      info['parameters'].query('dataset == "' + dataset + '"')[
                                                          'parameter'])))
    return day_df_wide


custom_import_functions = {'Sensor_py': mod_imported_Sensor_py_data,
                           'Skid': mod_imported_Skid_data,
                           'SampLog': mod_imported_SampLog_data,
                           'NWL': mod_imported_NWL_data,
                           'UO': mod_imported_UO_data}


# ### Data Import Functions
# Data import
def import_data(dataset):
    dataset_all_days = []
    for folder in range(1, len(info['datasets'].query('dataset == "' + dataset + '"')) + 1):
        # print(dataset + ": Folder " + str(folder))

        # Shorthand info:
        dataset_f_info = info['datasets'].query('dataset == "' + dataset + '" & folder == "' + str(folder) + '"')
        # File format to import
        f_file_import = select_file_importer(dataset, dataset_f_info)

        # Custom pre-import functions
        if dataset in custom_preimport_functions:
            dataset_supp_data = custom_preimport_functions[dataset](dataset_f_info)
        else:
            dataset_supp_data = ""
        # Import files
        for filename in tqdm(os.listdir(str(dataset_f_info['data_folder_path'][dataset])), desc="Open files to import"):
            if re.search(dataset_f_info['file_pat'][dataset], filename) and not filename.startswith('.'):
                day_df = file_import_handler(dataset, dataset_f_info, filename, dataset_supp_data, f_file_import)
                # Keep if longer than 0 lines
                if len(day_df) > 0:
                    # Keep if within DateTime range in Info file
                    if max(day_df['DateTime']) >= min(info['charts']['chart_range_start']) and min(
                            day_df['DateTime']) <= max(info['charts']['chart_range_end']):
                        dataset_all_days.append(day_df)

    # Create one dataframe from all days data
    dataset_all_data = pd.concat(dataset_all_days, axis=0, ignore_index=True)
    dataset_all_data.sort_values(by=['DateTime'], inplace=True)
    dataset_all_data = dataset_all_data.reset_index(drop=True)

    dataset_all_data = dataset_all_data[dataset_all_data['DateTime'] >= min(info['charts']['chart_range_start'])]
    dataset_all_data = dataset_all_data[dataset_all_data['DateTime'] <= max(info['charts']['chart_range_end'])]

    return dataset_all_data


# File Import functions
def select_file_importer(dataset, dataset_f_info):
    if re.search("xls", dataset_f_info['file_pat'][dataset], re.IGNORECASE):
        return file_import_functions["xls"]
    elif re.search("csv", dataset_f_info['file_pat'][dataset], re.IGNORECASE):
        return file_import_functions["csv"]
    elif re.search("txt", dataset_f_info['file_pat'][dataset], re.IGNORECASE):
        return file_import_functions["txt"]
    else:
        raise ValueError("Unknown file pattern!")


def file_import_xls(dataset, dataset_f_info, filename):
    df = pd.read_excel("".join([str(dataset_f_info['data_folder_path'][dataset]), filename]),
                       skiprows=dataset_f_info['skiprows'][dataset])
    return df


def file_import_csv(dataset, dataset_f_info, filename):
    df = pd.read_csv("".join([str(dataset_f_info['data_folder_path'][dataset]), filename]),
                     skiprows=dataset_f_info['skiprows'][dataset])
    return df


def file_import_txt(dataset, dataset_f_info, filename):
    df = pd.read_csv("".join([str(dataset_f_info['data_folder_path'][dataset]), filename]),
                     sep="\t", skiprows=dataset_f_info['skiprows'][dataset])
    return df


file_import_functions = {'xls': file_import_xls,
                         'csv': file_import_csv,
                         'txt': file_import_txt}


def file_import_handler(dataset, dataset_f_info, filename, dataset_supp_data, f_file_import):
    day_df = f_file_import(dataset, dataset_f_info, filename)
    if len(day_df) > 0:
        # Name parameter columns
        day_df = day_df.rename(columns=dict(zip(info['parameters'].query('dataset == "' + dataset + '"')['code'],
                                                info['parameters'].query('dataset == "' + dataset + '"')['parameter'])))
        # Delete unit rows
        if not pd.isna(dataset_f_info['Del_unit_rows'][dataset]):
            day_df = day_df.drop(0).reset_index()
        ##Load custom data mod function?
        if dataset in custom_import_functions:
            day_df = custom_import_functions[dataset](day_df, filename, dataset_supp_data, dataset_f_info, dataset)

        #Choose parameters included in parameters sheet
        selected_pars = list(info['parameters'].query('dataset == "' + dataset + '"')['parameter'].values)
        day_df = day_df[['DateTime'] + selected_pars].copy() #Copy prevents hidden chain indexing later

        #Make floats
        for col in selected_pars:
            try:
                day_df.loc[:,col] = day_df.loc[:,col].astype(float)
            except:
                pass
        
        # Add blank row between files - to be implemented
        # if len(day_df) > 0:
        #    if not pd.isna(dataset_f_info['Add_blank_rows'][dataset]):
        #        day_df = day_df.append(pd.Series(), ignore_index=True)

    return day_df





In [66]:
# ### Data Import
selected_cols = [s for s in info['parameters'].columns.to_list() if "selected" in s]
selected = info['parameters'][selected_cols].isin([1]).any(axis=1)

selected_cols_ave = [s for s in info['parameters_ave'].columns.to_list() if "selected" in s]
selected_ave = info['parameters_ave'][selected_cols_ave].isin([1]).any(axis=1)

selected_datasets = list(selected[selected].index.unique().values) + list(selected_ave[selected_ave].index.unique().values)
selected_datasets = [x for i, x in enumerate(selected_datasets) if i == selected_datasets.index(x)]

del selected_cols, selected_cols_ave

dataset_data = []
for dataset in tqdm(selected_datasets, desc = "Import data from each dataset"):
    dataset_data.append(import_data(dataset))

Open files to import: 100%|██████████| 2/2 [00:00<00:00, 34.08it/s]

Open files to import:   0%|          | 0/4 [00:00<?, ?it/s]Urban Observatory data up-to-date!

Import data from each dataset: 100%|██████████| 5/5 [00:24<00:00,  4.86s/it]


In [68]:
dataset = 'Sensor_py'
# Data import
dataset_all_days = []
folder = 1
# Shorthand info:
dataset_f_info = info['datasets'].query('dataset == "' + dataset + '" & folder == "' + str(folder) + '"')
# File format to import
f_file_import = select_file_importer(dataset, dataset_f_info)

# Custom pre-import functions
if dataset in custom_preimport_functions:
    dataset_supp_data = custom_preimport_functions[dataset](dataset_f_info)
else:
    dataset_supp_data = ""
for filename in tqdm(os.listdir(str(dataset_f_info['data_folder_path'][dataset])), desc="Open files to import"):
    if re.search(dataset_f_info['file_pat'][dataset], filename) and not filename.startswith('.'):
        day_df = f_file_import(dataset, dataset_f_info, filename)
        if len(day_df) > 0:
            # Name parameter columns
            day_df = day_df.rename(columns=dict(zip(info['parameters'].query('dataset == "' + dataset + '"')['code'],
                                                    info['parameters'].query('dataset == "' + dataset + '"')['parameter'])))
            # Delete unit rows
            if not pd.isna(dataset_f_info['Del_unit_rows'][dataset]):
                day_df = day_df.drop(0).reset_index()
            ##Load custom data mod function?
            day_df['DateTime'] = day_df['Date'] + " " + day_df['Time']
            day_df['DateTime'] = pd.to_datetime(day_df['DateTime'], format='%d/%m/%y %H:%M:%S')
            day_df['DateTime'] = day_df['DateTime'].dt.tz_localize('UTC')

            # Current density
            day_df.iloc[:, day_df.columns.str.contains("__C")] = day_df.iloc[:, day_df.columns.str.contains("__C")] / (
                    math.pi * 0.6 ** 2 * 10)
            # Sum Current density
            day_df.iloc[:, day_df.columns.str.contains("SUM__C")] = day_df.iloc[:, day_df.columns.str.contains("SUM__C")] / 4

            # Sensible tilt (up = positive)
            day_df['SENSOR_TILT'] = -day_df['SENSOR_TILT']

           

Open files to import: 100%|██████████| 2/2 [00:00<00:00, 78.95it/s]


In [69]:
day_df

,Date,Time,SENSOR_pH,TEMP_OS,SENSOR_CONDUCTIVITY,BES_A1__V,BES_A2__V,BES_A3__V,BES_A4__V,TEMP_HOSE_A,...,BES_C3__C,BES_C4__C,BES_A_SUM__V,BES_B_SUM__V,BES_C_SUM__V,BES_A_SUM__C,BES_B_SUM__C,BES_C_SUM__C,REFRESH_WAIT,DateTime
0,09/04/20,10:21:14,7.143036,13.424844,970.489502,123.686670,64.672873,111.365108,78.939945,36.880495,...,3.827273,3.583088,378.664596,475.940086,379.637351,4.193550,5.270835,4.204323,250,2020-04-09 10:21:14+00:00
1,09/04/20,10:22:14,7.143019,13.435362,961.170898,126.280683,66.942635,115.580379,80.236951,42.833708,...,3.999638,3.683635,389.040649,489.882906,393.580171,4.308460,5.425245,4.358733,190,2020-04-09 10:22:14+00:00
2,09/04/20,10:23:16,7.143019,13.395898,975.869568,126.929186,66.942635,116.228882,81.533958,47.794718,...,3.999638,3.697999,391.634662,496.367939,394.552926,4.337188,5.497064,4.369506,128,2020-04-09 10:23:16+00:00
3,09/04/20,10:24:16,7.142791,13.465779,985.639954,126.929186,66.942635,115.256128,81.858210,47.794718,...,3.942183,3.669271,390.986158,495.070933,390.986158,4.330006,5.482701,4.330006,68,2020-04-09 10:24:16+00:00
4,09/04/20,10:25:18,7.144160,13.395038,976.029480,126.280683,66.618383,115.256128,80.885455,48.786920,...,3.927820,3.654907,389.040649,493.773926,390.661907,4.308460,5.468337,4.326415,296,2020-04-09 10:25:18+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,09/04/20,12:26:20,7.139498,13.803379,966.054871,135.035477,67.266886,123.686670,82.506713,52.755728,...,4.373097,4.344370,408.495747,520.038309,445.460433,4.523917,5.759204,4.933285,40,2020-04-09 12:26:20+00:00
103,09/04/20,12:27:22,7.138814,13.796021,961.389343,135.683981,67.591138,124.659425,82.506713,49.779122,...,4.416189,4.401825,410.441256,522.632322,449.675704,4.545463,5.787931,4.979968,278,2020-04-09 12:27:22+00:00
104,09/04/20,12:28:22,7.139042,13.783340,975.292786,135.359729,67.266886,124.335173,81.858210,56.724537,...,4.387461,4.387461,408.819998,522.308070,447.730194,4.527508,5.784340,4.958422,218,2020-04-09 12:28:22+00:00
105,09/04/20,12:29:24,7.138814,13.776778,981.307739,135.035477,66.942635,123.686670,82.506713,46.802516,...,4.373097,4.387461,408.171495,520.038309,446.433187,4.520326,5.759204,4.944058,156,2020-04-09 12:29:24+00:00


In [70]:
filename = os.listdir(str(dataset_f_info['data_folder_path'][dataset]))[0]
print(re.search(dataset_f_info['file_pat'][dataset], filename) and not filename.startswith('.'))

True


In [71]:
# Data import
dataset_all_days = []
for folder in range(1, len(info['datasets'].query('dataset == "' + dataset + '"')) + 1):
    # print(dataset + ": Folder " + str(folder))

    # Shorthand info:
    dataset_f_info = info['datasets'].query('dataset == "' + dataset + '" & folder == "' + str(folder) + '"')
    # File format to import
    f_file_import = select_file_importer(dataset, dataset_f_info)

    # Custom pre-import functions
    if dataset in custom_preimport_functions:
        dataset_supp_data = custom_preimport_functions[dataset](dataset_f_info)
    else:
        dataset_supp_data = ""
    # Import files
    for filename in tqdm(os.listdir(str(dataset_f_info['data_folder_path'][dataset])), desc="Open files to import"):
        if re.search(dataset_f_info['file_pat'][dataset], filename) and not filename.startswith('.'):
            day_df = file_import_handler(dataset, dataset_f_info, filename, dataset_supp_data, f_file_import)
            # Keep if longer than 0 lines
            if len(day_df) > 0:
                # Keep if within DateTime range in Info file
                if max(day_df['DateTime']) >= min(info['charts']['chart_range_start']) and min(
                        day_df['DateTime']) <= max(info['charts']['chart_range_end']):
                    dataset_all_days.append(day_df)

# Create one dataframe from all days data
dataset_all_data = pd.concat(dataset_all_days, axis=0, ignore_index=True)
dataset_all_data.sort_values(by=['DateTime'], inplace=True)
dataset_all_data = dataset_all_data.reset_index(drop=True)

dataset_all_data = dataset_all_data[dataset_all_data['DateTime'] >= min(info['charts']['chart_range_start'])]
dataset_all_data = dataset_all_data[dataset_all_data['DateTime'] <= max(info['charts']['chart_range_end'])]

return dataset_all_data

def file_import_handler(dataset, dataset_f_info, filename, dataset_supp_data, f_file_import):
    day_df = f_file_import(dataset, dataset_f_info, filename)
    if len(day_df) > 0:
        # Name parameter columns
        day_df = day_df.rename(columns=dict(zip(info['parameters'].query('dataset == "' + dataset + '"')['code'],
                                                info['parameters'].query('dataset == "' + dataset + '"')['parameter'])))
        # Delete unit rows
        if not pd.isna(dataset_f_info['Del_unit_rows'][dataset]):
            day_df = day_df.drop(0).reset_index()
        ##Load custom data mod function?
        if dataset in custom_import_functions:
            day_df = custom_import_functions[dataset](day_df, filename, dataset_supp_data, dataset_f_info, dataset)

        #Choose parameters included in parameters sheet
        selected_pars = list(info['parameters'].query('dataset == "' + dataset + '"')['parameter'].values)
        day_df = day_df[['DateTime'] + selected_pars].copy() #Copy prevents hidden chain indexing later

        #Make floats
        for col in selected_pars:
            try:
                day_df.loc[:,col] = day_df.loc[:,col].astype(float)
            except:
                pass
        
        # Add blank row between files - to be implemented
        # if len(day_df) > 0:
        #    if not pd.isna(dataset_f_info['Add_blank_rows'][dataset]):
        #        day_df = day_df.append(pd.Series(), ignore_index=True)

    return day_df



def mod_imported_Sensor_data(day_df, filename, dataset_supp_data, dataset_f_info, dataset="Sensor"):
    day_df['DateTime'] = day_df['Date'] + " " + day_df['Time']
    day_df['DateTime'] = pd.to_datetime(day_df['DateTime'], format='%d/%m/%Y %H:%M:%S')
    day_df['DateTime'] = day_df['DateTime'].dt.tz_localize('UTC')

    # Make on/off events integer binary numbers 1/0
    # on_events = ["^ON$", "^RUNNING$", "^CW$", "^AUTO$"]
    # off_events = ["^OFF$", "^STOP$", "^CCW$", "^MANUAL$"]

    # all_rgx = re.compile("|".join(on_events + off_events))
    # on_rgx = re.compile("|".join(on_events))
    # off_rgx = re.compile("|".join(off_events))

    # Sensor_events = {'ON': 1,
    #                 'RUNNING': 1,
    #                 'CW': 1,
    #                 'AUTO': 1,
    #                 'OFF': 0,
    #                 'STOP': 0,
    #                 'CCW': 0,
    #                 'MANUAL': 0}

    # string_cols = (day_df.applymap(type) == str).all(0).values
    # for col in day_df.iloc[:, string_cols]:
    #    if day_df[col].str.contains(all_rgx).any():
    #        day_df[col] = day_df[col].replace(on_rgx, '1', regex=True)
    #        day_df[col] = day_df[col].replace(off_rgx, '0', regex=True)
    #        day_df[col] = day_df[col].astype(int)
    # for col in day_df.iloc[:, string_cols]:
    # if day_df.iloc[:, string_cols].str.contains(all_rgx).any():
    # day_df[[i for i in list(day_df.columns) if i not in ['DateTime', 'Date', 'Time']]] = \
    # day_df[[i for i in list(day_df.columns) if i not in ['DateTime', 'Date', 'Time']]].replace(Sensor_events).apply(pd.to_numeric)
    # day_df.iloc[:, string_cols] = day_df.astype(int)

    # Current density
    day_df.iloc[:, day_df.columns.str.contains("__C")] = day_df.iloc[:, day_df.columns.str.contains("__C")] / (
            math.pi * 0.6 ** 2 * 10)
    # Sum Current density
    day_df.iloc[:, day_df.columns.str.contains("SUM__C")] = day_df.iloc[:, day_df.columns.str.contains("SUM__C")] / 4

    # Sensible tilt (up = positive)
    day_df['SENSOR_TILT'] = -day_df['SENSOR_TILT']

    return day_df

Open files to import: 100%|██████████| 2/2 [00:00<00:00, 45.31it/s]


SyntaxError: 'return' outside function (<ipython-input-71-7fcdcd8e87f3>, line 35)